In [1]:
import numpy as np



In [250]:
class ANN:
    def __init__(self, layer_size= [50,1], lr= 0.001, momentum= 0, batch_size= 5, init_seed = None, nb_epoch= 300):
        self.x = None
        self.y = None
        self.lr = lr
        self.momentum = momentum
        self.layer_size = layer_size
        self.weights = []
        self.biases = []
        self.velocity = []
        self.output_layer = []
        self.batch_size = batch_size
        self.nb_epoch = nb_epoch
        
        if init_seed != None:
            np.random.seed(init_seed)
            
        for i, layer in enumerate(layer_size):
            if i < (len(layer_size) - 1):
                self.weights.append(np.random.rand(layer_size[i], layer_size[i+1]))
                self.velocity.append(np.zeros((layer_size[i], layer_size[i+1])))
            self.biases.append(np.random.rand(layer_size[i]))
        
    def _sigmoid(self, x, derivative= False):
        sigmoid =  1 / (1 + np.exp(-x))
        if derivative:
            return sigmoid * (1 - sigmoid)
        else:
            return sigmoid
    
    def fit(self, x, y):
        self.x = np.array(x)
        self.y = np.array(y)
        self.weights = [np.random.rand(self.x.shape[1], self.layer_size[0])] + self.weights
        self.velocity = [np.zeros((self.x.shape[1], self.layer_size[0]))] + self.velocity
        
        for i in range(self.nb_epoch):
            loss = 0
            for idx in range(0, self.x.shape[0], self.batch_size):
                data_in = self.x[idx : min(idx + self.batch_size, self.x.shape[0])]
                data_target = self.y[idx : min(idx + self.batch_size, self.x.shape[0])]
                out = self._feed_forward(data_in)
                loss += np.sum((out-data_target)**2)
                self._backprop(data_in, data_target)
            print("Epoch {}, loss: {}".format(i, loss))
    
    def _feed_forward(self, input_data= []):
        self.output_layer = []
        in_layer = input_data
        for weight, bias in zip(self.weights, self.biases):
            out_layer = self._sigmoid(np.dot(in_layer, weight) + bias)
            self.output_layer.append(out_layer)
            
            in_layer = out_layer
        return out_layer
        
    def _backprop(self, data_in, data_target):
        for i, w in reversed(list(enumerate(self.weights))):
            if i == len(self.weights) - 1:
                out_d = 2 * (data_target - self.output_layer[i]) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
            elif i == 0:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(data_in.T, out_d) + self.velocity[i] * self.momentum
            else:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
            
            self.velocity[i] = d_weight
        
        for i, w in enumerate(self.weights):
            self.weights[i] += self.velocity[i]
            
    
    def predict(self, data):
        return self._feed_forward(data)
    
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases

In [251]:
ann = ANN(layer_size=[4, 1], momentum= 0.005, lr= 0.01, batch_size= 4, nb_epoch= 100)
x = [[0,1], [0,0], [1,0], [1,1]]
y = [[0], [0], [1], [1]]


ann.fit(x, y)
ann.predict([[0,0], [0,0.1]])

Epoch 0, loss: 1.7573472554572893
Epoch 1, loss: 1.3285761692853921
Epoch 2, loss: 1.0802555026694367
Epoch 3, loss: 1.0689068813426559
Epoch 4, loss: 1.0646589523547962
Epoch 5, loss: 1.0604960374867103
Epoch 6, loss: 1.0563153576366997
Epoch 7, loss: 1.0520561288540802
Epoch 8, loss: 1.0476542434366976
Epoch 9, loss: 1.0430401115370098
Epoch 10, loss: 1.0381368478283115
Epoch 11, loss: 1.0328581163795159
Epoch 12, loss: 1.0271056521722193
Epoch 13, loss: 1.0207664264736285
Epoch 14, loss: 1.013709437621643
Epoch 15, loss: 1.0057821365893644
Epoch 16, loss: 0.9968065447993975
Epoch 17, loss: 0.9865752003275927
Epoch 18, loss: 0.9748471914019823
Epoch 19, loss: 0.9613447199968375
Epoch 20, loss: 0.9457509026213731
Epoch 21, loss: 0.9277098775514775
Epoch 22, loss: 0.9068307539150195
Epoch 23, loss: 0.8826974845721463
Epoch 24, loss: 0.854887286284429
Epoch 25, loss: 0.8230005727231485
Epoch 26, loss: 0.7867051482216159
Epoch 27, loss: 0.7457960692675086
Epoch 28, loss: 0.70026939771495

array([[0.02528667],
       [0.02484969]])